##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# XNNPACK를 통해 기기 내 추론 잘라내기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_for_on_device_inference"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

[XNNPACK](https://github.com/google/XNNPACK)를 통해 장치 내 추론의 대기 시간을 개선하기 위한 Keras 가중치 잘라내기 가이드에 오신 것을 환영합니다.

이 가이드는 새로 도입된 `tfmot.sparsity.keras.PruningPolicy` API 사용을 보여주며 [XNNPACK 희소 추론](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference)를 사용하여 최신 CPU에서 대부분의 컨볼루셔널 모델을 가속화하는 데 어떻게 사용될 수 있는지 설명합니다.

이 가이드는 모델 생성 프로세스의 다음과 같은 단계를 다룹니다.

- 밀집 기준 구축 및 훈련하기
- 잘라내기로 모델 미세 조정하기
- TFLite로 변환하기
- 기기 내 벤치마크

이 가이드는 잘라내기로 미세 조정하기에 대한 모범 사례를 다루지 않습니다. 이 주제에 대한 더 자세한 정보를 보려면 [종합 가이드](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)를 확인하시기 바랍니다.

## 설정

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

%load_ext tensorboard

## 밀집 모델 구축 및 훈련하기

[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) 데이터세트의 분류 작업을 위한 간단한 기준 CNN을 구축하고 훈련합니다.

In [ ]:
# Load CIFAR10 dataset.
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True,
    with_info=True,
)

# Normalize the input image so that each pixel value is between 0 and 1.
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.image.convert_image_dtype(image, tf.float32), label

# Load the data in batches of 128 images.
batch_size = 128
def prepare_dataset(ds, buffer_size=None):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.cache()
  if buffer_size:
    ds = ds.shuffle(buffer_size)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

ds_train = prepare_dataset(ds_train,
                           buffer_size=ds_info.splits['train'].num_examples)
ds_val = prepare_dataset(ds_val)
ds_test = prepare_dataset(ds_test)

# Build the dense baseline model.
dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Compile and train the dense model for 10 epochs.
dense_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

dense_model.fit(
  ds_train,
  epochs=10,
  validation_data=ds_val)

# Evaluate the dense model.
_, dense_model_accuracy = dense_model.evaluate(ds_test, verbose=0)

## 희소 모델 구축하기

[종합 가이드](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)의 지침을 사용하여, 잘라내기(즉, `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` 정책)를 통해 기기 내 가속화를 목표로 하는 매개변수가 있는 `tfmot.sparsity.keras.prune_low_magnitude` 함수를 적용합니다.

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs.
end_epoch = 5

num_iterations_per_epoch = len(ds_train)
end_step =  num_iterations_per_epoch * end_epoch

# Define parameters for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.25,
                                                               final_sparsity=0.75,
                                                               begin_step=0,
                                                               end_step=end_step),
      'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

# Try to apply pruning wrapper with pruning policy parameter.
try:
  model_for_pruning = prune_low_magnitude(dense_model, **pruning_params)
except ValueError as e:
  print(e)

호출 `prune_low_magnitude`는 메시지 `Could not find a GlobalAveragePooling2D layer with keepdims = True in all output branches`를 포함한 `ValueError`를 초래합니다. 이 메시지는 정책 `tfmot.sparsity.keras.PruneForLatencyOnXNNPack`을 사용한 잘라내기를 지원하지 않는다는 것을 표시하며 명확하게 해당 레이어 `GlobalAveragePooling2D`에 매개변수 `keepdims = True`가 필요하다는 것을 나타냅니다. 이를 고치고 `prune_low_magnitude` 함수를 다시 적용해 봅시다.

In [ ]:
fixed_dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims=True),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Use the pretrained model for pruning instead of training from scratch.
fixed_dense_model.set_weights(dense_model.get_weights())

# Try to reapply pruning wrapper.
model_for_pruning = prune_low_magnitude(fixed_dense_model, **pruning_params)

`prune_low_magnitude` 호출이 오류 없이 완료되었다는 것은 모델이 `tfmot.sparsity.keras.PruneForLatencyOnXNNPack`를<br> 완전히 지원하며  [XNNPACK Sparse inference](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference)을 사용해 가속화 가능하다는 의미입니다.

### 회소 모델 미세 조정하기

[잘라내기 예시](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras.md)에 따라, 밀집 모델의 가중치를 사용하여 희소 모델을 미세 조정합니다. 25%(25%의 가중치가 0으로 설정됩니다)의 희소성으로 모델 미세 조정을 시작하여 75%의 희소성으로 종료합니다.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

model_for_pruning.fit(
  ds_train,
  epochs=15,
  validation_data=ds_val,
  callbacks=callbacks)

# Evaluate the dense model.
_, pruned_model_accuracy = model_for_pruning.evaluate(ds_test, verbose=0)

print('Dense model test accuracy:', dense_model_accuracy)
print('Pruned model test accuracy:', pruned_model_accuracy)

로그는 레이어별로 희소성의 진행률을 보여줍니다.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

잘라내기로 미세 조정 후, 테스트 정확도는 밀집 모델과 비교하여 약간 개선(43%에서 44%) 되었습니다. [TFLite 기준](https://www.tensorflow.org/lite/performance/measurement)을 사용하여 기기 내 대기 시간을 비교해 봅시다.

## 최신 변환 및 벤치마킹

TFLite에 잘라내기 된 모델을 변환하려면, `PruneLowMagnitude` 래퍼를 `strip_pruning` 함수를 통해 원래 레이어로 교체해야 합니다. 또한, 잘라내기 된 모델(`model_for_pruning`)은 대부분 가중치가 0이기 때문에, 결과가 된 TFLite 모델을 효과적으로 저장하기 위해 최적화 `tf.lite.Optimize.EXPERIMENTAL_SPARSITY`를 적용할 수 있습니다. 이 최적화 플래그는 밀집 모델에 필요하지 않습니다.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(dense_model)
dense_tflite_model = converter.convert()

_, dense_tflite_file = tempfile.mkstemp('.tflite')
with open(dense_tflite_file, 'wb') as f:
  f.write(dense_tflite_model)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

[TFLite 모델 벤치마킹 도구](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark)의 지침에 따라, 도구를 구축하고 안드로이드 기기에 밀집 및 잘라내기 된 TFLite 모델과 함께 업로드하고 기기에서 두 모델을 벤치마크합니다.

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/dense_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/pruned_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

Pixel 4에서의 벤치마크는 밀집 모델의 경우 평균 추론 시간이 *17us*였고 잘라내기 된 모델의 경우 *12us*였습니다. 기기 내 벤치마크는 심지어 소규모 모델에서도 명확한 **5us** 또는 대기 시간 **30%** 향상을 보여주었습니다. 경험상 [MobileNetV3](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v3) 또는 [EfficientNet-lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite)에 기반한 대규모 모델도 유사한 성능 향상을 보여줍니다. 속도 향상은 전체 모델에 대한 1x1 컨볼루션의 상대적 기여도에 따라 달라집니다.


## 결론

이 튜토리얼에서, TF MOT API 및 XNNPack에 의해 도입된 새로운 기능을 사용하여 더욱 빠른 기기 내 성능을 위한 희소 모델을 생성하는 방법을 보여드립니다. 이러한 희소 모델은 밀집이 동등한 모델보다 규모가 작고 빠르면서도 품질을 유지하거나 심지어 능가합니다.

이 새로운 기능은 기기 내에서 모델을 배포할 때 특히 중요할 수 있어 사용해 보는 것이 좋습니다.